In [8]:
# In[1]: Import necessary libraries
import os
import numpy as np
import pickle
import pandas as pd
from tqdm import tqdm

# In[2]: Function to load poses from a .txt file
def load_poses(file_path):
    poses = []
    with open(file_path, 'r') as f:
        lines = f.readlines()
        for line in lines:
            pose = np.fromstring(line, sep=' ').reshape(3, 4)
            # Add the last row of homogeneous coordinates. [0, 0, 0, 1]
            pose = np.vstack([pose, [0, 0, 0, 1]])
            poses.append(pose)
    return poses

# In[3]: Function to compute the relative transformation between two poses
def compute_relative_pose(pose1, pose2):
    pose1_inv = np.linalg.inv(pose1)
    relative_pose = np.dot(pose2, pose1_inv)
    return relative_pose

# In[4]: Define paths and settings
base_path = '/project/bli4/autoai/joyang/GeoTransformer/data/Rellis'
metadata_path = os.path.join(base_path, 'metadata')
train_scenes = ['00000', '00001', '00002']
val_scene = '00003'
test_scene = '00004'

train_metadata = []
val_metadata = []
test_metadata = []

# In[5]: Function to process a single scene and save the GT data to metadata
def process_scene(scene_dir, metadata_list):
    scene_path = os.path.join(base_path, 'sequences', scene_dir, 'poses.txt')
    
    if not os.path.isfile(scene_path):
        return
    
    poses = load_poses(scene_path)
    interval = 50
    step = 5
    
    for i in range(0, len(poses) - interval, step):
        frame0 = i
        frame1 = i + interval
        if frame1 >= len(poses):
            break
        pose1 = poses[frame0]
        pose2 = poses[frame1]
        relative_pose = compute_relative_pose(pose1, pose2)
        
        metadata_list.append({
            'seq_id': int(scene_dir),
            'frame0': frame0,
            'frame1': frame1,
            'transform': relative_pose,
            'pcd0': f'downsampled/{scene_dir}/{str(frame0).zfill(6)}.npy',
            'pcd1': f'downsampled/{scene_dir}/{str(frame1).zfill(6)}.npy'
        })

# In[6]: Process and save training data
for scene_dir in train_scenes:
    process_scene(scene_dir, train_metadata)

train_pkl_path = os.path.join(metadata_path, 'train.pkl')
with open(train_pkl_path, 'wb') as f:
    pickle.dump(train_metadata, f)

print(f"Saved training metadata to {train_pkl_path}")

# In[7]: Process and save validation data
process_scene(val_scene, val_metadata)

val_pkl_path = os.path.join(metadata_path, 'val.pkl')
with open(val_pkl_path, 'wb') as f:
    pickle.dump(val_metadata, f)

print(f"Saved validation metadata to {val_pkl_path}")

# In[8]: Process and save testing data
process_scene(test_scene, test_metadata)

test_pkl_path = os.path.join(metadata_path, 'test.pkl')
with open(test_pkl_path, 'wb') as f:
    pickle.dump(test_metadata, f)

print(f"Saved testing metadata to {test_pkl_path}")


Saved training metadata to /project/bli4/autoai/joyang/GeoTransformer/data/Rellis/metadata/train.pkl
Saved validation metadata to /project/bli4/autoai/joyang/GeoTransformer/data/Rellis/metadata/val.pkl
Saved testing metadata to /project/bli4/autoai/joyang/GeoTransformer/data/Rellis/metadata/test.pkl
